In [1]:
import numpy as np
from h5py import File
import scipy.io #Used to load the OCTAVE *.mat files
import numpy as np
import sys
sys.path.append ('../src')
from NeuralNetwork import NNClassifier
from ML_utils import softmax,sigmoid,UTIL_formatY,backward_prop,backpropagation
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix,classification_report




In [2]:
MNIST_data = File("data/MNISTdata.hdf5", 'r')
x_train = np.float32(MNIST_data['x_train'][:])
y_train = np.int32(np.array(MNIST_data['y_train'][:, 0])).reshape(-1, 1)
x_test  = np.float32(MNIST_data['x_test'][:])
y_test  = np.int32(np.array(MNIST_data['y_test'][:, 0])).reshape(-1, 1)
MNIST_data.close()

# stack together for next step
X = np.vstack((x_train, x_test))
print (X.shape)
y = np.vstack((y_train, y_test))
print (y.shape)

# one-hot encoding
digits = 10
examples = y.shape[0]
y = y.reshape(1, examples)
Y_new = np.eye(digits)[y.astype('int32')]
Y_new = Y_new.T.reshape(digits, examples)


# number of training set
m = 60000
m_test = X.shape[0] - m
X_train, X_test = X[:m].T, X[m:].T
Y_train, Y_test = Y_new[:, :m], Y_new[:, m:]


# shuffle training set
shuffle_index = np.random.permutation(m)
X_train, Y_train = X_train[:, shuffle_index], Y_train[:, shuffle_index]

(70000, 784)
(70000, 1)


In [4]:
opt = {'n_h':64,'n_x':784,'epochs':10, 'batch_size':64 , 'beta':0.9,'lr':0.5}

# initialization
params = {"W1": np.random.randn(opt['n_h'], opt['n_x']) * np.sqrt(1. / opt['n_x']),
          "b1": np.zeros((opt['n_h'], 1)) * np.sqrt(1. / opt['n_x']),
          "W2": np.random.randn(digits, opt['n_h']) * np.sqrt(1. / opt['n_h']),
          "b2": np.zeros((digits, 1)) * np.sqrt(1. / opt['n_h'])}
print (params['W1'].shape)
print (params['b1'].shape)
print (params['W2'].shape)
print (params['b2'].shape)

(64, 784)
(64, 1)
(10, 64)
(10, 1)


In [5]:
T1 = np.copy(np.hstack(( params['b1'],params['W1'])))
print (T1.shape)
T2 = np.copy(np.hstack(( params['b2'],params['W2'],)))
print (T2.shape)


(64, 785)
(10, 65)


In [6]:

nn_config={'n_a1':784,'n_a2':64,'n_a3':10 }   # Configuración de red NN , input layer , hidder layers , output layer
sgd_dict =   {'steps':10,'learning_rate':0.5,'mini_batch_size':2**8}
opt_dict =   {'maxiter':100,'algorithm' : 'TNC'}
activ = {'activation_a2':sigmoid,'activation_a3':softmax}
method = 'miniBatchGD'
nn = NNClassifier(optimization=method,bias=True,nn_config=nn_config,activ=activ,debug=False,kargs=sgd_dict)
print (nn.thetas['Theta1'].shape)
print (nn.thetas['Theta2'].shape)
nn.thetas['Theta1'] = T1
nn.thetas['Theta2'] = T2

(64, 785)
(10, 65)


In [7]:
def sigmoid(z):
    """
    sigmoid activation function.

    inputs: z
    outputs: sigmoid(z)
    """
    s = 1. / (1. + np.exp(-z))
    return s
def compute_loss(Y, Y_hat):
    """
    compute loss function
    """
    L_sum = np.sum(np.multiply(Y, np.log(Y_hat)))
    m = Y.shape[1]
    L = -(1./m) * L_sum

    return L
def feed_forward(X, params):
    """
    feed forward network: 2 - layer neural net

    inputs:
        params: dictionay a dictionary contains all the weights and biases

    return:
        cache: dictionay a dictionary contains all the fully connected units and activations
    """
    cache = {}

    # Z1 = W1.dot(x) + b1
    cache["Z1"] = np.matmul(params["W1"], X) + params["b1"]
    
    # A1 = sigmoid(Z1)
    cache["A1"] = sigmoid(cache["Z1"])

    # Z2 = W2.dot(A1) + b2
    cache["Z2"] = np.matmul(params["W2"], cache["A1"]) + params["b2"]

    # A2 = softmax(Z2)
    cache["A2"] = np.exp(cache["Z2"]) / np.sum(np.exp(cache["Z2"]), axis=0)
#    cache["A2"] = sigmoid(cache["Z2"])

    return cache
def back_propagate(X, Y, params, cache, m_batch):
    """
    back propagation

    inputs:
        params: dictionay a dictionary contains all the weights and biases
        cache: dictionay a dictionary contains all the fully connected units and activations

    return:
        grads: dictionay a dictionary contains the gradients of corresponding weights and biases
    """
    # error at last layer
    dZ2 = cache["A2"] - Y

    # gradients at last layer (Py2 need 1. to transform to float)
    dW2 = (1. / m_batch) * np.matmul(dZ2, cache["A1"].T)
    db2 = (1. / m_batch) * np.sum(dZ2, axis=1, keepdims=True)

    # back propgate through first layer
    dA1 = np.matmul(params["W2"].T, dZ2)
    dZ1 = dA1 * sigmoid(cache["Z1"]) * (1 - sigmoid(cache["Z1"]))

    # gradients at first layer (Py2 need 1. to transform to float)
    dW1 = (1. / m_batch) * np.matmul(dZ1, X.T)
    db1 = (1. / m_batch) * np.sum(dZ1, axis=1, keepdims=True)

    grads = {"dW1": dW1, "db1": db1, "dW2": dW2, "db2": db2}

    return grads

In [8]:


batches=10
for i in range(opt['epochs']):

    # shuffle training set
    permutation = np.random.permutation(X_train.shape[1])
    X_train_shuffled = X_train[:, permutation]
    Y_train_shuffled = Y_train[:, permutation]

    for j in range(batches):
        #np.testing.assert_allclose (params['W1'],nn.thetas['Theta1'][:,1:])
        #np.testing.assert_allclose (params['W2'],nn.thetas['Theta2'][:,1:])
        #np.testing.assert_allclose (params['b1'],nn.thetas['Theta1'][:,0:1])
        #np.testing.assert_allclose (params['b2'],nn.thetas['Theta2'][:,0:1])

        # get mini-batch
        begin = j * opt['batch_size']
        end = min(begin + opt['batch_size'], X_train.shape[1] - 1)
        X = X_train_shuffled[:, begin:end]
        Y = Y_train_shuffled[:, begin:end]
        m_batch = end - begin

        nnX_train = np.copy(X.reshape (X.shape[1],X.shape[0]))
        nnY_train = np.copy(Y.reshape (Y.shape[1],Y.shape[0]))

        # forward and backward
        cache = feed_forward(X, params)
        #prediction,nncache = nn.forward_prop (X.T)
        #np.testing.assert_allclose (prediction.T,cache['A2'])
        grads = back_propagate(X, Y, params, cache, m_batch)
        #delta,nngrads = nn.backward_prop (X.T,Y.T)
        #np.testing.assert_allclose (grads['dW1'],nngrads['grad1'][:,1:])
        #np.testing.assert_allclose (grads['dW2'],nngrads['grad2'][:,1:])
        #np.testing.assert_allclose (grads['db1'],nngrads['grad1'][:,0:1])
        #np.testing.assert_allclose (grads['db2'],nngrads['grad2'][:,0:1])

        # with momentum (optional)
        dW1 = grads ['dW1']
        db1 = grads ['db1']
        dW2 = grads ['dW2']
        db2 = grads ['db2']
        
        dW1 = (opt['beta'] * dW1 + (1. - opt['beta']) * grads["dW1"])
        db1 = (opt['beta'] * db1 + (1. - opt['beta']) * grads["db1"])
        dW2 = (opt['beta'] * dW2 + (1. - opt['beta']) * grads["dW2"])
        db2 = (opt['beta'] * db2 + (1. - opt['beta']) * grads["db2"])
    
        # gradient descent
        params["W1"] = params["W1"] - opt['lr'] * dW1
        params["b1"] = params["b1"] - opt['lr'] * db1
        params["W2"] = params["W2"] - opt['lr'] * dW2
        params["b2"] = params["b2"] - opt['lr'] * db2
        #nn._updateThetas (nngrads,opt['lr'])
        #np.testing.assert_allclose (params['W1'],nn.thetas['Theta1'][:,1:])
        
    # forward pass on training set
    cache = feed_forward(X_train, params)
    #prediction,_ = nn.forward_prop (X_train.T)
    train_loss = compute_loss(Y_train, cache["A2"])
    #cost = nn.costFunction (X_train.T,Y_train.T)
    # forward pass on test set
    cache = feed_forward(X_test, params)
    test_loss = compute_loss(Y_test, cache["A2"])
    print("Epoch {}: training loss = {}, test loss = {}".format(
        i + 1, train_loss, test_loss))

Epoch 1: training loss = 2.0218250609556985, test loss = 2.011223814678099
Epoch 2: training loss = 1.7102189761749014, test loss = 1.696897230081713
Epoch 3: training loss = 1.3683998968670448, test loss = 1.3485243002761087
Epoch 4: training loss = 1.1725721642429656, test loss = 1.1520234590994696
Epoch 5: training loss = 0.9738696628673802, test loss = 0.9547045261974605
Epoch 6: training loss = 0.8338735937558812, test loss = 0.8136594693331972
Epoch 7: training loss = 0.7549405748926117, test loss = 0.7366832410100067
Epoch 8: training loss = 0.6753378511379609, test loss = 0.6555891625418807
Epoch 9: training loss = 0.6380658612306976, test loss = 0.6217727929076532
Epoch 10: training loss = 0.5945605636415341, test loss = 0.5767862518812692


In [9]:
from sklearn.metrics import multilabel_confusion_matrix,classification_report
result = feed_forward (X_test,params)
prediction = np.argmax(result['A2'],axis=0)
print (prediction[0:5])

print(f"Classification report for classifier :\n"
      f"{classification_report(y_test,prediction)}\n")

[7 2 1 0 4]
Classification report for classifier :
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       980
           1       0.88      0.98      0.93      1135
           2       0.88      0.85      0.86      1032
           3       0.81      0.89      0.85      1010
           4       0.84      0.91      0.87       982
           5       0.79      0.79      0.79       892
           6       0.84      0.93      0.88       958
           7       0.86      0.89      0.88      1028
           8       0.92      0.68      0.78       974
           9       0.93      0.72      0.81      1009

    accuracy                           0.86     10000
   macro avg       0.87      0.86      0.86     10000
weighted avg       0.87      0.86      0.86     10000




In [10]:
nn_config={'n_a1':784,'n_a2':64,'n_a3':10 }   # Configuración de red NN , input layer , hidder layers , output layer
sgd_dict =   {'steps':10,'learning_rate':0.5,'mini_batch_size':2**8}
opt_dict =   {'maxiter':100,'algorithm' : 'TNC'}
activ = {'activation_a2':sigmoid,'activation_a3':softmax}
method = 'miniBatchGD'

nn = NNClassifier(optimization=method,bias=True,nn_config=nn_config,activ=activ,debug=False,kargs=sgd_dict)
costs = nn.optimize (X_train.T,Y_train.T,l2_lambda=0.0)
prediction,_ = nn.forward_prop (X_train.T)
result = np.argmax(prediction,axis=1).reshape(-1,1)
y = np.argmax(Y_train.T,axis=1).reshape(-1,1)

accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )

test_predicted,_ = nn.forward_prop (X_test.T)
result = np.argmax(test_predicted,axis=1).reshape(-1,1)
y = np.argmax(Y_test.T,axis=1).reshape(-1,1)
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")

LR = 0.5:   0%|                                                                                 | 0/10 [00:00<?, ?it/s]

2.322800996180674
2.266844381352002
2.3107006217585777
2.2528169122018706
2.2057541790358397
2.232184788446172
2.205018508837408
2.1605155746034654
2.141990049801877
2.120040625873781
2.1200491908514527
2.106963943400876
2.0754486873584748
2.046403581526258
2.0358133326304477
1.987277643617945
1.9697988197549563
1.9461876259555349
1.9195639097330903
1.892749450345127
1.8336827683995214
1.8038979017886203
1.7616315039266093
1.7444211046654325
1.6991274095232543
1.7123114966980313
1.6131497219109592
1.6354988031947328
1.6807678293845818
1.5690733181739493
1.5222687763787603
1.4756857674629562
1.4257438548267938
1.45124698690632
1.3735592866256916
1.3868300058212415
1.3769888611933552
1.3246445481283309
1.3437109829697667
1.3257263495410763
1.227428320218114
1.3090642808936908
1.1806073418956466
1.1641427000649522
1.1927423806970672
1.144070497307058
1.069366436306938
1.1949820238849163
1.0344541851530662
1.1319579257592105
1.038276258883156
1.0869915494389155
1.0213695205384563
1.0184408

LR = 0.5:  10%|███████▎                                                                 | 1/10 [00:05<00:53,  5.99s/it]

0.42518107692595475
0.41736619289506094
0.45499341607595656
0.44619056131207663
0.3813118722343781
0.32328375700676265
0.35607395815691983
0.4002944152184428
0.3881947685203758
0.42352833667098244
0.4027822320561893
0.3621665668667763
0.4034878053515891
0.43581631309199326
0.45479665563593064
0.48457226304003975
0.39405648496791124
0.389085451248616
0.3462064297294652
0.4049451229444731
0.37785819767783135
0.35260240817283206
0.43560483703706354
0.40829201567427814
0.36217076373122087
0.38240540165410714
0.3207373511284597
0.3968869102130884
0.5094668718934318
0.34891940456505455
0.37000822020797797
0.39889385315422377
0.3137471682338975
0.37552129714986227
0.38873127408197444
0.4039512261846447
0.4222916715983464
0.338024271055102
0.3422808494769496
0.4803555551610236
0.2893434555332869
0.4501377961559145
0.3086545993846558
0.36633517868452203
0.366619892091593
0.31814621956610467
0.33086579908368013
0.41627024638151755
0.32055199645624705
0.4328406606999476
0.3792623269808262
0.47736

LR = 0.5:  20%|██████████████▌                                                          | 2/10 [00:12<00:48,  6.04s/it]

...

LR = 0.5:  90%|█████████████████████████████████████████████████████████████████▋       | 9/10 [00:51<00:05,  5.68s/it]


0.18417880524198332
0.14216515693144055
0.18805244057139434
0.15731522129589293
0.1858104628209651
0.2054302937397435
0.170019017270078
0.11826224170557874
0.13020954635269655
0.1458497879771059
0.17230069522788638
0.1410587066497589
0.17798000374037704
0.1383749102851048
0.18379920831417834
0.17356542602245006
0.22070889556477097
0.23642694838127598
0.17519240302217456
0.14546173346159091
0.16510349758697962
0.1757069660128216
0.16382836644809126
0.14196876737418027
0.2529520281055393
0.15194146492613803
0.16317194579764852
0.22014524818672218
0.12330329680561139
0.20166268521015324
0.24362196105111802
0.1397968508146389
0.1714455183461005
0.2040427796620653
0.09842511574095188
0.1846447891908724
0.19781465323766537
0.17915936208192948
0.192477458877566
0.12726148899338874
0.12277553540221803
0.26618029515498054
0.10489605366706058
0.20188127496170044
0.13621052116299318
0.1672717684912865
0.15739002910939204
0.11237473868341208
0.16958796659693864
0.2501100819848424
0.15351140535877

LR = 0.5: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:57<00:00,  5.76s/it]


0.1494667990178542
0.11817089745296028
0.1737202469789696
0.13467846135887626


Trainig set accuracy : 95.42333333333333
Test set accuracy : 95.11
Classification report for classifier <NeuralNetwork.NNClassifier object at 0x00000220915A39A0>:
              precision    recall  f1-score   support

           0       0.95      0.99      0.97       980
           1       0.98      0.98      0.98      1135
           2       0.94      0.96      0.95      1032
           3       0.94      0.95      0.94      1010
           4       0.94      0.95      0.95       982
           5       0.95      0.92      0.93       892
           6       0.95      0.96      0.95       958
           7       0.95      0.94      0.95      1028
           8       0.97      0.93      0.95       974
           9       0.94      0.93      0.93      1009

    accuracy                           0.95     10000
   macro avg       0.95      0.95      0.95     10000
weighted avg       0.95      0.95      0.95     10000




In [3]:
nn_config={'n_a1':784,'n_a2':128 ,'n_a3':64,'n_a4':25 , 'n_a5':10}   # Configuración de red NN , input layer , hidder layers , output layer
sgd_dict =   {'steps':50,'learning_rate':0.5,'mini_batch_size':2**8}
opt_dict =   {'maxiter':500,'algorithm' : 'TNC'}
activ = {'activation_a2':sigmoid,'activation_a3':sigmoid,'activation_a4':sigmoid,'activation_a5':softmax}
method = 'miniBatchGD'
if method == 'Optimize':
    midict = opt_dict
else:
    midict = sgd_dict
    
nn = NNClassifier(optimization=method,bias=True,nn_config=nn_config,activ=activ,debug=False,kargs=midict)
costs = nn.optimize (X_train.T,Y_train.T,l2_lambda=0.0)

prediction,_ = nn.forward_prop (X_train.T)
print (prediction.shape)
print (Y_train.shape)

result = np.argmax(prediction,axis=1).reshape(-1,1)
y = np.argmax(Y_train.T,axis=1).reshape(-1,1)

accuracy = np.mean(y==result) * 100
print ('Trainig set accuracy :' , accuracy  )

test_predicted,_ = nn.forward_prop (X_test.T)
result = np.argmax(test_predicted,axis=1).reshape(-1,1)
y = np.argmax(Y_test.T,axis=1).reshape(-1,1)
accuracy = np.mean(y==result) * 100
print ('Test set accuracy :' , accuracy  )

print(f"Classification report for classifier {nn}:\n"
      f"{classification_report(y,result)}\n")

LR = 0.5: 100%|████████████████████████████████████████████████████████████████████████| 50/50 [06:09<00:00,  7.39s/it]


(60000, 10)
(10, 60000)
Trainig set accuracy : 99.49833333333333
Test set accuracy : 97.35000000000001
Classification report for classifier <NeuralNetwork.NNClassifier object at 0x00000261D07DAE80>:
              precision    recall  f1-score   support

           0       0.99      0.98      0.98       980
           1       0.98      0.99      0.98      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.97      0.97      1010
           4       0.97      0.97      0.97       982
           5       0.98      0.97      0.97       892
           6       0.98      0.97      0.98       958
           7       0.97      0.96      0.97      1028
           8       0.95      0.98      0.96       974
           9       0.96      0.97      0.97      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000




In [4]:
print (prediction.shape)
print (X_train.shape)
print (Y_train.shape)

(60000, 10)
(784, 60000)
(10, 60000)
